In [86]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, model_selection,tree, linear_model, neighbors, naive_bayes, ensemble 
from matplotlib import pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from path import Path


## Visualizing the data and checking for correlations

In [87]:
# Load the viral_data.csv dataset.
filepath = Path('viral_data.csv')
viral_df = pd.read_csv(filepath, index_col=0)
print(viral_df.shape)
viral_df.head()

(200, 7)


,State,Year,total_pop,Uninsured_%,PC_Income,GDP,Flu_%
0,Alabama,2016.0,4863525.0,9.1,39536.0,191523,0.237400
1,Alaska,2016.0,741456.0,14.0,56302.0,53289,0.143367
2,Arizona,2016.0,6941072.0,10.0,40801.0,291259,0.322227
3,Arkansas,2016.0,2989918.0,7.9,40385.0,113490,0.096424
4,California,2016.0,39167117.0,7.3,58048.0,2519133,0.112745


In [88]:
# Generate our categorical variable list
viral_list = viral_df.dtypes[viral_df.dtypes == "object"].index.tolist()
# # Check the number of unique values in each column
# loans_df[loans_cat].nunique()

In [89]:
viral_df = viral_df[viral_df['Year'] == 2016]
viral_df.drop(columns=['Year'], inplace=True)

In [90]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(viral_df[viral_list]))
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(viral_list)
encode_df.head()

,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,State_Colorado,State_Connecticut,State_Delaware,State_Florida,State_Georgia,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
# Merge one-hot encoded features and drop the originals
viral_df = viral_df.merge(encode_df,left_index=True, right_index=True)
viral_df = viral_df.drop(viral_list,1)
print(viral_df.shape)
viral_df.head()

(50, 55)


,total_pop,Uninsured_%,PC_Income,GDP,Flu_%,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,4863525.0,9.1,39536.0,191523,0.237400,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,741456.0,14.0,56302.0,53289,0.143367,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6941072.0,10.0,40801.0,291259,0.322227,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2989918.0,7.9,40385.0,113490,0.096424,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,39167117.0,7.3,58048.0,2519133,0.112745,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
viral_df.head()

,total_pop,Uninsured_%,PC_Income,GDP,Flu_%,State_Alabama,State_Alaska,State_Arizona,State_Arkansas,State_California,...,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,4863525.0,9.1,39536.0,191523,0.237400,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,741456.0,14.0,56302.0,53289,0.143367,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6941072.0,10.0,40801.0,291259,0.322227,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2989918.0,7.9,40385.0,113490,0.096424,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,39167117.0,7.3,58048.0,2519133,0.112745,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
# Remove flu_% target from features data
y = np.array(viral_df['Flu_%'])
X = viral_df.drop('Flu_%', axis = 1)

In [96]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [97]:
# Check dat for correctness
print('Training Features Shape:', X_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (40, 54)
Testing Features Shape: (10, 54)
Training Labels Shape: (40,)
Testing Labels Shape: (10,)


In [99]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

In [100]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
predictions

array([0.11542943, 0.16740061, 0.20970811, 0.10765416, 0.18939382,
       0.07166639, 0.12974572, 0.19731243, 0.20448355, 0.05424917])

In [101]:
# Claculating the root mean square
from sklearn.metrics import mean_squared_error
y_true = y_test
y_pred = predictions
mean_squared_error(y_true, y_pred, squared=False)

0.42534087989983643

In [102]:
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

-0.14108953726302986

In [103]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [105]:
# Create a random forest classifier.
rf_model = RandomForestRegressor(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

In [106]:
# Calculating the root mean square
from sklearn.metrics import mean_squared_error
y_true = y_test
mean_squared_error(y_true, y_pred, squared=False)

0.425373418765404

In [107]:
# Calculating the root square
from sklearn.metrics import r2_score
r2_score(y_true, y_pred)

-0.14126413218693257